In [ ]:
import pandas as pd

ACC = pd.read_csv("admin092503_202009251622_ACCELEROMETER.txt")
GRV = pd.read_csv("admin092503_202009251622_GRAVITY.txt")

In [ ]:
GRV_XYZ = GRV.iloc[:,-3:]
GRV_XYZ.columns = ["gx","gy","gz"]
ACC_XYZ = ACC.iloc[:,-3:]
ACC_XYZ.columns = ["ax","ay","az"]

In [ ]:
sum_df = pd.concat([GRV_XYZ, ACC_XYZ], axis=1)

result = pd.DataFrame()
result["X_gap"] = sum_df["gx"] - sum_df["ax"]
result["Y_gap"] = sum_df["gy"] - sum_df["ay"]
result["Z_gap"] = sum_df["gz"] - sum_df["az"]

result

In [ ]:
GRV_time = pd.DataFrame()
GRV_time = GRV["Time"]

result = pd.concat([GRV_time, result], axis=1)
result

In [ ]:
result.to_csv("Gravity_ACC_gap.csv")

# SENSOR DATA TEST

In [1]:
import json
import pandas as pd
import pandas.io.sql as sql
from sqlalchemy.engine import URL
from sqlalchemy import create_engine, event
from pandas.core.algorithms import value_counts
import pyodbc
import time, datetime
import numpy as np
from azure.storage.blob.blockblobservice import BlockBlobService
from tempfile import NamedTemporaryFile
import pytz

In [13]:
def get_file(filename,container_name):
    blob_service = BlockBlobService(
            account_name='tokmadatastorage',
            account_key='')
    # 파일을 담을 임시 디렉토리 생성
    local_file = NamedTemporaryFile()
    # 블롭에서 Stream으로 데이터를 받아 Local_file 임시 디렉토리에 저장
    blob_service.get_blob_to_stream(container_name, filename, stream=local_file, 
    max_connections=2)
    # 저장된 파일의 0번 Memory 지정
    local_file.seek(0)

    # 지정된 메모리 리턴 
    return local_file

In [14]:
def gen_timestamp_df(result_df):
    stamp_list = []
    for i in result_df["SENSING_TIME"]:
        s_time = i[:19]
        stamp = int(time.mktime(datetime.datetime.strptime(str(s_time),'%Y-%m-%d %H:%M:%S').timetuple()))-32400
        stamp_list.append(stamp)
    stamp_df = pd.DataFrame(stamp_list)
    stamp_df.columns = ["TIMESTAMP_VAL"]
    result_df = pd.concat([result_df, stamp_df], axis=1)
    
    return result_df

In [15]:
def get_json_HRM(container_name, blob_name):
    data = str(get_file(blob_name,container_name).read())[2:-1]
    data = data.split("\\n")[1:]
    u_name = blob_name.split("/")[0]
    result = [] 
    for index in data:
        obj = index.split(",")
        result.append(obj)

    u_name = blob_name.split("/")[0]
    result_df = pd.DataFrame(result).dropna().sort_values(by=0).drop([1],axis=1)
    result_df.columns = ["SENSING_TIME", "SENSOR_VAL"]
    result_df.sort_values(["SENSING_TIME"], inplace=True)
    result_df = gen_timestamp_df(result_df)
    result_df["SENSOR_TYPE"] = "HRM"
    result_df["SENIOR_ID"] = u_name
    KST = pytz.timezone('Asia/Seoul')
    result_df["CREATE_DATE"] = str(datetime.datetime.now(tz=KST))[:19]
    result_df["CREATE_USER"] = "AF"
    result_df = result_df[["SENIOR_ID","SENSOR_TYPE","TIMESTAMP_VAL","SENSOR_VAL","SENSING_TIME","CREATE_DATE","CREATE_USER"]]
                                  
    return result_df

In [16]:
def get_json_step(container_name, blob_name):
    data = str(get_file(blob_name,container_name).read())[2:-1]
    data = data.split("\\n")[1:]
    u_name = blob_name.split("/")[0]
    result = [] 
    for index in data:
        obj = index.split(",")
        result.append(obj)

    u_name = blob_name.split("/")[0]
    result_df = pd.DataFrame(result).dropna().sort_values(by=0).drop([1],axis=1)
    result_df.columns = ["SENSING_TIME", "SENSOR_VAL"]
    result_df = result_df.astype({"SENSOR_VAL":int})
    result_df["SENSOR_VAL"] = result_df["SENSOR_VAL"].sum()
    result_df["SENSING_TIME"] = result_df["SENSING_TIME"].max()
    result_df = gen_timestamp_df(result_df)
    result_df["SENSOR_TYPE"] = "StepCount"
    result_df["SENIOR_ID"] = u_name
    KST = pytz.timezone('Asia/Seoul')
    result_df["CREATE_DATE"] = str(datetime.datetime.now(tz=KST))[:19]
    result_df["CREATE_USER"] = "AF"
    result_df = result_df[["SENIOR_ID","SENSOR_TYPE","TIMESTAMP_VAL","SENSOR_VAL","SENSING_TIME","CREATE_DATE","CREATE_USER"]]
    result_df = result_df.head(1)

    return result_df

In [17]:
container_name = "w-sensordata"
blob_name = "minisoft11/STEPCOUNTER_202112021830.txt"
df =  get_json_step(container_name=container_name, blob_name=blob_name)
df

Client-Request-ID=2ad68f04-5b27-11ec-80d6-000d3a8db9c6 Retry policy did not allow for a retry: Server-Timestamp=Sun, 12 Dec 2021 08:40:32 GMT, Server-Request-ID=823bfdb7-b01e-004a-6233-ef140a000000, HTTP status code=404, Exception=The specified blob does not exist. ErrorCode: BlobNotFound<?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.RequestId:823bfdb7-b01e-004a-6233-ef140a000000Time:2021-12-12T08:40:32.8994866Z</Message></Error>.


AzureMissingResourceHttpError: The specified blob does not exist. ErrorCode: BlobNotFound
<?xml version="1.0" encoding="utf-8"?><Error><Code>BlobNotFound</Code><Message>The specified blob does not exist.
RequestId:823bfdb7-b01e-004a-6233-ef140a000000
Time:2021-12-12T08:40:32.8994866Z</Message></Error>